In [ ]:
import os

import numpy as np

import pandas as pd
import cv2
import matplotlib.pyplot as plt

import urllib

In [ ]:
req = urllib.request.urlopen('url1')
arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
img1 = cv2.imdecode(arr, -1)

req = urllib.request.urlopen('url2')
arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
img2 = cv2.imdecode(arr, -1)

# Extract descriptors

In [ ]:
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

In [ ]:
sift = cv2.SIFT_create()

kp1, desp1 = sift.detectAndCompute(img1,None)
kp2, desp2 = sift.detectAndCompute(img2,None)

# Bruteforce matching

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

matches = bf.match(desp1,desp2)
matches = sorted(matches, key = lambda x:x.distance)


In [ ]:
img3 = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], img2, flags=4)


In [ ]:
plt.imshow(img3)

# FLANN based matching

In [ ]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary

flann = cv2.FlannBasedMatcher(index_params,search_params)


In [ ]:
matches = flann.knnMatch(desp1,desp2,k=2)

# ratio test
good = []
for i,(m,n) in enumerate(matches):
    if m.distance < 0.7*n.distance:
        good += [m]
        
good = sorted(good, key = lambda x:x.distance) 


In [ ]:
img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)


In [ ]:
plt.imshow(img3)

# Batch processing

In [ ]:
df = pd.read_csv('./input.csv', index_col=None)

In [ ]:
sift = cv2.SIFT_create()

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary

flann = cv2.FlannBasedMatcher(index_params,search_params)

In [ ]:
def resize_and_pad(image, new_size):
    h, w = image.shape[:2]
    ratio = float(new_size) / max(h,w)

    new_hw = tuple(int(ratio * x) for x in (h,w))
    image = cv2.resize(image, (new_hw[1], new_hw[0]))  # resize expects (w,h)

    delta_h = new_size - new_hw[0]
    delta_w = new_size - new_hw[1]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])

    # image = cv2.copyMakeBorder(image, 0, delta_h, 0, delta_w, cv2.BORDER_WRAP)
    return image


In [ ]:
n_thld50 = []
n_thld100 = []
n_thld150 = []
n_thld200 = []
for r in df.itertuples():
    try:
        req = urllib.request.urlopen(r.query_img_url)
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        img1 = cv2.imdecode(arr, -1)
        w = max(img1.shape[:2])
        if w < 640:
            img1 = cv2.resize(img1, (640, 640), interpolation=cv2.INTER_CUBIC)
            

        req = urllib.request.urlopen(r.comp_img_url)
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        img2 = cv2.imdecode(arr, -1)
        w = max(img2.shape[:2])
        if w < 640:
            img2 = cv2.resize(img2, (640, 640), interpolation=cv2.INTER_CUBIC)

        kp1, desp1 = sift.detectAndCompute(img1,None)
        kp2, desp2 = sift.detectAndCompute(img2,None)

        match_pairs = flann.knnMatch(desp1,desp2,k=2)

        # ratio test
        good = []
        for i,(m,n) in enumerate(match_pairs):
            if m.distance < 0.75*n.distance:
                good += [m]
    except Exception as e:
        print(str(e))
        good = []

    good = sorted(good, key = lambda x:x.distance) 
    
    tops = [m.distance for m in good if m.distance < 50]
    n_thld50 += [len(tops)]
    tops = [m.distance for m in good if m.distance < 100]
    n_thld100 += [len(tops)]
    tops = [m.distance for m in good if m.distance < 150]
    n_thld150 += [len(tops)]
    tops = [m.distance for m in good if m.distance < 200]
    n_thld200 += [len(tops)]


In [ ]:
df['count50'] = n_thld50
df['count100'] = n_thld100
df['count150'] = n_thld150
df['count200'] = n_thld200


In [ ]:
df.to_csv('./output.csv', index=None)